In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy.stats
import seaborn as sns

# enables concurrent editing of base.py
from importlib import reload
reload(base)

sns.set_style('ticks')
sns.set_context('talk',rc={'font.family': 'sans-serif', 'font.sans-serif':['Helvetica Neue']})

## Setup

- Load data
- Add metadata
- Draw gates
- Gate transfected cells

Result from this section: DataFrame `df` representing transfected cells.

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'collaborators'/'birnbaum_steph'/'2024-06-10 Galloway Exp 1'
output_path = rd.rootdir/'output'/'lenti-exp97'
cache_path = output_path/'data.gzip'

rd.plot.plot_well_metadata(base_path/'metadata.yaml')

In [ ]:
# Load data
data = pd.DataFrame()
if cache_path.is_file(): data = pd.read_parquet(cache_path)
else: 
    channels = ['FSC-A', 'SSC-A', 'FITC-A', 'PE-A', 'APC-A750-A']
    data = rd.flow.load_csv_with_metadata(base_path/'export', base_path/'metadata.yaml', columns=channels)
    data = data.rename({'FITC-A': 'mGL-A', 'PE-A': 'mRuby2-A', 'APC-A750-A': 'livedead-A'}, axis=1)
    
    data.to_parquet(rd.outfile(cache_path))
# Create dicts to specify colors/markers
metadata = base.get_metadata(rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct-metadata.xlsx')
metadata['TS'] = metadata['ts_kind']
metadata_dict = metadata.set_index('construct').to_dict('dict')
construct_palette = metadata_dict['color']
construct_markers = metadata_dict['markers']
display(data)

### Extremely basic analysis

In [ ]:
data['short_condition'] = data.construct.map({"pKG3097": "base", "RC130": "OL", "RC131": "CL"})

In [ ]:
mGL_gate = data[data.construct == 'UT']['mGL-A'].quantile(0.9999)
mRuby2_gate = data[data.construct == 'UT']['mRuby2-A'].quantile(0.9999)
livedead_gate = 2000

for channel, gate in zip(['mGL-A', 'mRuby2-A', 'livedead-A'], [mGL_gate, mRuby2_gate, livedead_gate]):
    sns.kdeplot(data[data[channel] > 0], x=channel, log_scale=True, hue='construct')
    plt.axvline(gate, color='k', ls=':')
    plt.show()

In [ ]:
infection_summary = data.groupby(['construct', 'moi'])['mRuby2-A'].apply(lambda s: np.mean(s > mRuby2_gate)).reset_index()
sns.stripplot(infection_summary, x='moi', y='mRuby2-A', hue='construct')
sns.despine()
plt.ylim([0,1])
plt.ylabel('Infection efficiency')
plt.title('Activated T-cells')
plt.xlabel('MOI')
plt.savefig(rd.outfile(rd.rootdir/'output'/'moi_data'/'t_cells.svg'), bbox_inches='tight')
plt.show()

In [ ]:
gated = data[(data['livedead-A'] < livedead_gate) & (data['mGL-A'] > mGL_gate) & (data['mRuby2-A'] > mRuby2_gate)].copy()

In [ ]:
n_quantiles = 20
transfection_bin = gated.groupby(['dox', 'construct', 'moi', 'short_condition']).apply(lambda df: pd.qcut(df['mGL-A'], n_quantiles, labels=np.linspace(0,1,n_quantiles, endpoint=False)), include_groups=False).reset_index().set_index('level_4')['mGL-A']
gated['transfection_bin'] = transfection_bin

In [ ]:
def compute_bin_gmeans(df):
    # Calculate the gmean by bin
    return df.loc[:, ('mGL-A', 'mRuby2-A')].apply(scipy.stats.gmean)
bin_gmeans = (gated.groupby(['dox', 'construct', 'moi', 'transfection_bin'], observed=True)
      .apply(compute_bin_gmeans, include_groups=False)
      .rename(columns={"mGL-A": "mGL-bin-gmean", "mRuby2-A": "mRuby2-bin-gmean"}).reset_index())
gated_with_bins = pd.merge(gated, bin_gmeans, on=['dox', 'construct', 'moi', 'transfection_bin'], validate="many_to_one")

In [ ]:
g = sns.FacetGrid(data=bin_gmeans[bin_gmeans.dox == 1000], col='construct', hue='moi', palette='viridis', margin_titles=True)
g.map(sns.scatterplot, 'mGL-bin-gmean', 'mRuby2-bin-gmean')
for ax in g.axes.flatten():
    ax.set_xscale('log')
    ax.set_yscale('log')
plt.savefig(rd.outfile(output_path/'raw_gmeans_log.svg'), bbox_inches='tight')
plt.show()

In [ ]:
n_quantiles = 100
df = data[(data['mGL-A']>0) & (data['mRuby2-A']>0)].copy()
transfection_bin = df.groupby(['dox', 'construct', 'moi', 'short_condition']).apply(lambda df: pd.qcut(df['mGL-A'], n_quantiles, labels=np.linspace(0,1,n_quantiles, endpoint=False)), include_groups=False).reset_index().set_index('level_4')['mGL-A']
df['transfection_bin'] = transfection_bin

bin_gmeans = (df.groupby(['dox', 'construct', 'moi', 'transfection_bin'], observed=True)
      .apply(compute_bin_gmeans, include_groups=False)
      .rename(columns={"mGL-A": "mGL-bin-gmean", "mRuby2-A": "mRuby2-bin-gmean"}).reset_index())
binned = pd.merge(df, bin_gmeans, on=['dox', 'construct', 'moi', 'transfection_bin'], validate="many_to_one")
gates = [df[df.construct == 'UT']['mGL-A'].quantile(q) for q in [0.99, 0.999, 0.9999, 0.99995, 0.99999]]

g = sns.FacetGrid(data=bin_gmeans[bin_gmeans.dox == 1000], col='construct', hue='moi', palette='viridis', margin_titles=True)
g.map(sns.scatterplot, 'mGL-bin-gmean', 'mRuby2-bin-gmean')
for ax in g.axes.flatten():
    ax.set_xscale('log')
    ax.set_yscale('log')
    for gate in gates: ax.axvline(gate)
plt.savefig(rd.outfile(output_path/'gating_mGL.svg'), bbox_inches='tight')
plt.show()